<a href="https://colab.research.google.com/github/adityasharma10699/demopygit/blob/main/LifeExpectancy_RFR%2BSVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 2.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from category_encoders import TargetEncoder
from sklearn.svm import SVR
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Importing dataset and examining it
dataset = pd.read_csv("/content/drive/MyDrive/AS & ML/Regression/LifeExpectancy.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset.head())
print(dataset.shape)
print(dataset.info())
print(dataset.describe())

# Converting Categorical features into Numerical features
dataset['Status'] = dataset['Status'].map({'Developed': 1, 'Developing':0})
print(dataset.info())

# # Plotting Correlation Heatmap
# corrs = dataset.corr()
# figure = ff.create_annotated_heatmap(
#     z=corrs.values,
#     x=list(corrs.columns),
#     y=list(corrs.index),
#     annotation_text=corrs.round(2).values,
#     showscale=True)
# figure.show()

# Dividing dataset into label and feature sets
X = dataset.drop(['Life expectancy'], axis = 1) # Features
Y = dataset['Life expectancy'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

#target Encoding
enc = TargetEncoder(cols=['Country']).fit(X,Y)
X = enc.transform(X)
print(X.info())

# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

# # Linear Regression with Regularization
# # Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter', along with the regularization parameter alpha using Grid Search
# sgdr = SGDRegressor(random_state = 1, penalty = 'elasticnet')
# grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100], 'l1_ratio': [0,0.25,0.5,0.75,1]}

# gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print("Best parameters: ", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("r2: ", best_result)

# Adj_r2 = 1-(1-best_result)*(1168-1)/(1168-19-1)
# print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''

# best_model = gd_sr.best_estimator_
# print("Intercept: ", best_model.intercept_)

# print(pd.DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

##################################################################################
# Implementing Random Forest Regression
# Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
grid_param = {'n_estimators': [10,20,30,40,50,100]}

gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(1579-1)/(1579-21-1)
print("Adjusted r2: ", Adj_r2)

'''
Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

where, n = number of observations in training data, p = number of features
'''

featimp = pd.Series(gd_sr.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(featimp)

# # # Selecting features with higher sifnificance and redefining feature set
# # X_ = dataset[['GrLivArea', 'YearBuilt', 'TotalBsmtSF', 'GarageArea']]

# # feature_scaler = StandardScaler()
# # X_scaled_ = feature_scaler.fit_transform(X_)

# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [50,100,150,200,250]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled_, Y)

# best_parameters = gd_sr.best_params_
# print("Best parameters: ", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("r2: ", best_result)

# Adj_r2 = 1-(1-best_result)*(1168-1)/(1168-21-1)
# print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''
####################################################################################
# Implementing Support Vector Regression
# Tuning the SVR parameters 'kernel', 'C', 'epsilon' and implementing cross-validation using Grid Search
svr = SVR()
grid_param = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [.001,.01, 0.1, 1, 10,100]}

gd_sr = GridSearchCV(estimator=svr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(1579-1)/(1579-21-1)
print("Adjusted r2: ", Adj_r2)

       Country  Year      Status  Life expectancy  Adult Mortality  \
0  Afghanistan  2015  Developing             65.0              263   
1  Afghanistan  2014  Developing             59.9              271   
2  Afghanistan  2013  Developing             59.9              268   
3  Afghanistan  2012  Developing             59.5              272   
4  Afghanistan  2011  Developing             59.2              275   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   BMI  \
0             62     0.01               71.279620         65.0     1154  19.1   
1             64     0.01               73.523580         62.0      492  18.6   
2             66     0.01               73.219240         64.0      430  18.1   
3             69     0.01               78.184220         67.0     2787  17.6   
4             71     0.01                7.097109         68.0     3013  17.2   

   under-five deaths  Polio  Total expenditure  Diphtheria  HIV/AIDS  \
0                 83